# Building Keras BYOC - Manual
### Libraries for Model

In [1]:
import pandas as pd
import numpy as np
import time
import shutil
import os, shutil
import random
import cv2
import math
import json
import urllib.request
import zipfile
import sagemaker
import glob

import keras
from keras.preprocessing.image import *
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Lambda, ELU
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Additional Libraries for Notebook

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display # Allows the use of display() for DataFrames

# Visualizations will be shown in the notebook.
%matplotlib inline

---
## Sample `Model.py`
### Helper Functions - Image Augmentation

In [3]:
def download(url):
    """
    Helper function to download individual file from given url.
    
    Arguments:
    url -- full URL of the file to download
    
    Returns:
    filename -- downloaded file name
    """
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
    return filename

In [4]:
def load_image(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))

In [5]:
def crop(image):
    """
    Crop the image (removing the sky at the top and the car front at the bottom)
    """
    return image[60:-25, :, :] # remove the sky and the car front

In [6]:
def resize(image):
    """
    Resize the image to the input shape used by the network model
    """
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)

In [7]:
def rgb2yuv(image):
    """
    Convert the image from RGB to YUV (This is what the NVIDIA model does)
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

In [8]:
def preprocess(image):
    """
    Combine all preprocess functions into one
    """
    image = crop(image)
    image = resize(image)
    image = rgb2yuv(image)
    return image

In [9]:
def choose_image(data_dir, center, left, right, steering_angle):
    """
    Randomly choose an image from the center, left or right, and adjust
    the steering angle.
    """
    choice = np.random.choice(3)
    if choice == 0:
        return load_image(data_dir, left), steering_angle + 0.2
    elif choice == 1:
        return load_image(data_dir, right), steering_angle - 0.2
    return load_image(data_dir, center), steering_angle

In [10]:
def random_flip(image, steering_angle):
    """
    Randomly flipt the image left <-> right, and adjust the steering angle.
    """
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle
    return image, steering_angle

In [11]:
def random_translate(image, steering_angle, range_x, range_y):
    """
    Randomly shift the image virtially and horizontally (translation).
    """
    trans_x = range_x * (np.random.rand() - 0.5)
    trans_y = range_y * (np.random.rand() - 0.5)
    steering_angle += trans_x * 0.002
    trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, trans_m, (width, height))
    return image, steering_angle

In [12]:
def distort(image):
    ''' 
    method for adding random distortion to dataset images, including random brightness adjust, and a random
    vertical shift of the horizon position
    '''
    new_img = image.astype(float)
    # random brightness - the mask bit keeps values from going beyond (0,255)
    value = np.random.randint(-28, 28)
    if value > 0:
        mask = (new_img[:,:,0] + value) > 255 
    if value <= 0:
        mask = (new_img[:,:,0] + value) < 0
    new_img[:,:,0] += np.where(mask, 0, value)
    # random shadow - full height, random left/right side, random darkening
    h,w = new_img.shape[0:2]
    mid = np.random.randint(0,w)
    factor = np.random.uniform(0.6,0.8)
    if np.random.rand() > .5:
        new_img[:,0:mid,0] *= factor
    else:
        new_img[:,mid:w,0] *= factor
    # randomly shift horizon
    h,w,_ = new_img.shape
    horizon = 2*h/5
    v_shift = np.random.randint(-h/8,h/8)
    pts1 = np.float32([[0,horizon],[w,horizon],[0,h],[w,h]])
    pts2 = np.float32([[0,horizon+v_shift],[w,horizon+v_shift],[0,h],[w,h]])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    new_img = cv2.warpPerspective(new_img,M,(w,h), borderMode=cv2.BORDER_REPLICATE)
    return new_img.astype(np.uint8)

In [13]:
# Radomly Adjust Brightness
def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    """
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

In [14]:
def augument(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    """
    Generate an augumented image and adjust steering angle.
    (The steering angle is associated with the center image)
    """
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    image = distort(image)
    image = random_brightness(image)
    return image, steering_angle

In [15]:
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            # argumentation
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augument(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center) 
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers

### Helper Functions - Model Training

In [16]:
def load_data(data_dir, test_size):
    """
    Load training data and split it into training and validation set
    """
    data_df = pd.read_csv(os.path.join(data_dir, 'driving_log.csv'))

    X = data_df[['center', 'left', 'right']].values
    y = data_df['steering'].values

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=0)

    return X_train, X_valid, y_train, y_valid

### Model Graph

In [17]:
def build_model(input_shape):
    """
    Comma.ai model
    """
    model = Sequential()
    model.add(
        Lambda(
            lambda x: x/127.55 -1,
            input_shape=input_shape
        )
    )
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
    model.add(ELU())
    model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
    model.add(ELU())
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))
    model.summary()
    return model

---
## Training
### Load Training Data

In [18]:
# To download and extract Sample Data
file = download('https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip')

# Extract the file
with zipfile.ZipFile(file) as zf:
    zf.extractall()

In [19]:
# Directory for the training data
data_dir = 'data'

# Train/Test Split = 90/10
test_size = 0.1

# Load the data
X_train, X_valid, y_train, y_valid = load_data(data_dir, test_size)

# Image parameters
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

### Load Model

In [20]:
# Build the model
model = build_model(IMAGE_SHAPE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 50, 16)        3088      
_________________________________________________________________
elu_1 (ELU)                  (None, 17, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 25, 32)         12832     
_________________________________________________________________
elu_2 (ELU)                  (None, 9, 25, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 13, 64)         51264     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4160)              0         
__________

### Train Model

In [21]:
# Save model Checpoint
checkpoint = ModelCheckpoint(
    'model-{epoch:03d}.h5',
    verbose=0,
    save_best_only=1,
    mode='auto'
)

In [22]:
# Compile model
model.compile(loss='mean_squared_error', optimizer=Adam(lr=1.0e-4))

In [23]:
# Fit the model
model.fit_generator(
    batch_generator(
        data_dir,
        X_train,
        y_train,
        16,
        True
    ),
    2000, # Samples per epoch
    8, # Number of epochs
    max_q_size=1,
    validation_data=batch_generator(
        data_dir,
        X_valid,
        y_valid,
        16,
        False
    ),
    nb_val_samples=len(X_valid),
    callbacks=[checkpoint],
    verbose=1
)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 2000, 8, validation_data=<generator..., callbacks=[<keras.ca..., verbose=1, validation_steps=804, max_queue_size=1)`


Epoch 1/8
2000/2000 [==============================] - 155s 78ms/step - loss: 0.0321 - val_loss: 0.0127
Epoch 2/8
2000/2000 [==============================] - 154s 77ms/step - loss: 0.0250 - val_loss: 0.0136
Epoch 3/8
2000/2000 [==============================] - 153s 77ms/step - loss: 0.0234 - val_loss: 0.0145
Epoch 4/8
2000/2000 [==============================] - 153s 77ms/step - loss: 0.0226 - val_loss: 0.0123
Epoch 5/8
2000/2000 [==============================] - 153s 77ms/step - loss: 0.0213 - val_loss: 0.0127
Epoch 6/8
2000/2000 [==============================] - 154s 77ms/step - loss: 0.0214 - val_loss: 0.0125
Epoch 7/8
2000/2000 [==============================] - 154s 77ms/step - loss: 0.0216 - val_loss: 0.0116
Epoch 8/8
2000/2000 [==============================] - 153s 77ms/step - loss: 0.0203 - val_loss: 0.0114


### Save Model

In [24]:
# remove any existing files from testing
!rm /tmp/model.*

In [25]:
# Simulate code that will go into SageMaker
MODEL_NAME = 'model.h5'
MODEL_PATH = '/tmp' # Will be '/opt/ml/model' in SageMaker

# Simulate environmental variable in SageMAker Train
model_dir = MODEL_PATH 

def save(model, model_dir):
    print("Saving the trained model ...")
    model_path = os.path.join(model_dir)
    files = list(glob.glob(os.path.join('./', '*.h5')))
    
    # Find the best model weights
    best = max(files)
    
    # Rename the best weights
    os.rename(best, MODEL_NAME)
    
    # Move to model_dir
    shutil.move('./model.h5', model_dir+'/')
    
    # Save model graph to `.json`
    model_json = model.to_json()
    with open(model_dir+'/model.json', 'w') as outfile:
        json.dump(model_json, outfile)

In [26]:
save(model, model_dir)

Saving the trained model ...


In [27]:
!ls -la /tmp/

total 25980
drwxrwxrwt  5 root       root         147456 Jul  3 20:31 .
dr-xr-xr-x 26 root       root           4096 Jul  3 17:51 ..
drwxr-xr-x  2 role-agent role-agent     4096 Jul  3 17:51 hsperfdata_role-agent
drwxrwxrwt  2 root       root           4096 Jul  3 17:50 .ICE-unix
drwxr-xr-x  3 role-agent role-agent     4096 Jul  3 17:51 jetty-0.0.0.0-9081-role-proxy-agent.war-_-any-9129967830017298108.dir
-rw-rw-r--  1 ec2-user   ec2-user   26427216 Jul  3 20:31 model.h5
-rw-rw-r--  1 ec2-user   ec2-user       4431 Jul  3 20:31 model.json
